# 데이터 전처리 작업 및 데이터 저장

이 코드는 이미지 데이터의 데이터 전처리를 수행하고, 결과를 CSV 파일로 저장하는 작업을 수행합니다.   
데이터 전처리 과정은 얼굴을 감지하고 잘라내며, 이미지를 그레이스케일로 변환하여 저장됩니다. 

In [1]:
# !pip install tqdm

In [2]:
import os
from glob import glob
import json
import cv2
import numpy as np
import pandas as pd
from random import shuffle
from tqdm import tqdm
CROP_SIZE = (224,224) #사진크기 지정


In [3]:
def create_path(path):
    if not os.path.exists(path):
        os.mkdir(path)
        print(f'Create {path}.')
    else:
        print(f'{path} already exists.')

In [4]:
class Datapreprocess:
    def __init__(self):
        """
        데이터 전처리 클래스의 초기화 메서드입니다.
        """
        self.img_folder_path = "D:\ML_pj\Training"
        self.data_lst = []  # 데이터 목록을 저장할 리스트
        self.img = None  # 이미지 데이터를 저장할 변수
        self.X_position = (0, 0)  # 얼굴 영역의 X 좌표 범위
        self.Y_position = (0, 0)  # 얼굴 영역의 Y 좌표 범위
        self.curr_emotion = ''  # 현재 감정 레이블
        self.new_file_path = ""  # 새로운 파일 경로
        self.img_path = ""
        self.file_name = ""
    def _save_crop_img(self):
        img = self.img.copy()
        roi = img[
            int(self.Y_position[0]):int(self.Y_position[1]),
            int(self.X_position[0]):int(self.X_position[1]),
        ]
        img = cv2.resize(roi, CROP_SIZE, interpolation=cv2.INTER_CUBIC)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        self.img = img

    def _detect_face(self,info):
        self.img_path = os.path.join(self.img_folder_path,self.curr_emotion,info['filename'])
        self.img = cv2.imread(self.img_path)
        self.X_position = info['annot_A']['boxes']['minX'], info['annot_A']['boxes']['maxX']
        self.Y_position = info['annot_A']['boxes']['minY'], info['annot_A']['boxes']['maxY']


    def work(self, info, emo):
        """
        데이터 전처리 작업을 수행하는 메서드입니다.

        :param img_path: 입력 이미지 파일의 경로
        :param emo: 해당 이미지의 감정 레이블
        :return: 데이터 전처리 성공 여부 (True 또는 False)
        """
        self.curr_emotion = emo
        self.file_name = info['filename']
        self._detect_face(info)
        self._save_crop_img()
        self.new_file_path = f"./{self.curr_emotion}/{self.file_name}"
        self.data_lst.append({"path": self.new_file_path, "label": self.curr_emotion})
        cv2.imwrite(self.new_file_path, self.img)
        df = pd.DataFrame(self.data_lst)
        df.to_csv(f"./csvs/{self.curr_emotion}.csv")

In [5]:
emotion_lst = ['angry','embarrassed']


datapreprocess = Datapreprocess()
curr_dir = os.getcwd()
create_path(os.path.join(curr_dir,'csvs'))
create_path(os.path.join(curr_dir,'errors'))
label_path = 'D:\ML_pj\Training\labels'


for emo in emotion_lst:
    print(emo)
    create_path(os.path.join(curr_dir,emo))
    datapreprocess.__init__()
    json_path = os.path.join(label_path,f"{emo}.json")
    with open(json_path,'r') as f:
        info_lst = json.load(f)
    for info in tqdm(info_lst):
        try:
            datapreprocess.work(info, emo)  # 각 이미지에 대해 데이터 전처리 작업을 수행합니다.
        except:
            with open(f'./errors/{emo}.txt','a')as f:
               f.write(str(info['filename'])+'\n') 

C:\Users\royal\Desktop\ML\project\csvs already exists.
C:\Users\royal\Desktop\ML\project\errors already exists.
angry
C:\Users\royal\Desktop\ML\project\angry already exists.


100%|██████████████████████████████████████████████████████████████████████████| 58396/58396 [3:23:40<00:00,  4.78it/s]


embarrassed
C:\Users\royal\Desktop\ML\project\embarrassed already exists.


100%|██████████████████████████████████████████████████████████████████████████| 58325/58325 [4:54:12<00:00,  3.30it/s]
